In [2]:
import csv
import os
from PIL import Image
import torch

base_dir = 'sample/images/train'

dataset_list = [os.path.join(base_dir,file) for file in os.listdir(base_dir) if file.endswith('.png')]
dataset_list.sort()
label_list = [label.replace('.png', '.txt').replace('images','labels') for label in dataset_list]
# Combine dataset_list and label_list into a single dataset
combined_dataset = list(zip(dataset_list, label_list))
combined_dataset

[('sample/images/train/OBJ00022_PS3_K3_NIA0078.png',
  'sample/labels/train/OBJ00022_PS3_K3_NIA0078.txt'),
 ('sample/images/train/OBJ00028_PS3_K3_NIA0078.png',
  'sample/labels/train/OBJ00028_PS3_K3_NIA0078.txt'),
 ('sample/images/train/OBJ00032_PS3_K3_AIDATA0585.png',
  'sample/labels/train/OBJ00032_PS3_K3_AIDATA0585.txt'),
 ('sample/images/train/OBJ00036_PS3_K3_AIDATA0585.png',
  'sample/labels/train/OBJ00036_PS3_K3_AIDATA0585.txt'),
 ('sample/images/train/OBJ00059_PS3_K3_AIDATA0587.png',
  'sample/labels/train/OBJ00059_PS3_K3_AIDATA0587.txt')]

In [55]:
import cv2
import os
import numpy as np
from PIL import Image
from tqdm import tqdm


def cluster_and_crop(image_path, k=3, size=1024, crop_num=43, threshold=0.7):
    # 이미지 로드 및 리사이즈
    img = Image.open(image_path)
    img = img.resize((size, size))
    
    # 이미지를 numpy 배열로 변환 (RGB 값 형태)
    img_np = np.array(img)
    
    # 이미지 데이터를 준비 (픽셀 수, 3) 형태로 변형
    img_data = img_np.reshape((-1, 3))
    img_data = np.float32(img_data)

    # K-Means 적용
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(img_data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # 클러스터링 결과를 이미지로 변환
    centers = np.uint8(centers)
    segmented_img = centers[labels.flatten()]
    segmented_img = segmented_img.reshape(img_np.shape)

    # 검은색에 가장 가까운 클러스터 찾기
    black_cluster_index = np.argmin(np.linalg.norm(centers, axis=1))  # 가장 작은 RGB 값이 검은색
    
    # crop 영역 추출
    num_patches = []
    crop_size = segmented_img.shape[1] // crop_num
    for i in range(crop_num):
        for j in range(crop_num):
            
            crop_img = segmented_img[i * crop_size:(i + 1) * crop_size, j * crop_size:(j + 1) * crop_size]

            # 검은색 클러스터 비율 확인
            cluster_0_value = centers[black_cluster_index]
            cluster_0_mask = np.all(crop_img == cluster_0_value, axis=-1)
            if np.mean(cluster_0_mask) > threshold:
                num_patches.append((i, j))

    
    return num_patches

def save_cropped_patches_as_numpy(image_path, 
                                  crop_size, 
                                  resize_size,
                                  cluster_img_size,
                                  cluster_threshold,
                                  save_dir):
    image_name = os.path.basename(image_path)
    print(f"Processing {image_name}")
    image = Image.open(image_path).convert('RGB')
    
    image_width, image_height = image.size
    
    num_patches = cluster_and_crop(image_path, 
                                   k=3, 
                                   size=cluster_img_size, 
                                   crop_num=(image_width + crop_size - 1) // crop_size, 
                                   threshold=cluster_threshold)
    
    for i, j in tqdm(num_patches):
        top_left_x = i * crop_size
        top_left_y = j * crop_size
        bottom_right_x = min(top_left_x + crop_size, image_width)
        bottom_right_y = min(top_left_y + crop_size, image_height)
        
        if bottom_right_x - top_left_x < crop_size:
            top_left_x = max(image_width - crop_size, 0)
            bottom_right_x = image_width
        if bottom_right_y - top_left_y < crop_size:
            top_left_y = max(image_height - crop_size, 0)
            bottom_right_y = image_height
        
        cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
        
        cropped_image = cropped_image.resize((resize_size, resize_size), resample=Image.Resampling.LANCZOS)
        
        crop_filename = f"{os.path.splitext(image_name)[0]}_{top_left_x}_{top_left_y}.png"
        crop_path = os.path.join(save_dir, crop_filename)
        cropped_image.save(crop_path, format="PNG")


    del image

crop_size = 256
resize_size = 1024
cluster_img_size = 1024
cluster_threshold = 0.1
target_dir = './target_dir'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

save_cropped_patches_as_numpy("./tasksample.png", 
                                          crop_size, 
                                          resize_size, 
                                          cluster_img_size,
                                          cluster_threshold,
                                          target_dir)

Processing tasksample.png


100%|██████████| 1560/1560 [06:52<00:00,  3.79it/s]


In [58]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

# 상대 좌표를 절대 좌표로 변환하는 함수
def convert_to_absolute(label, img_size):
    h, w = img_size
    abs_coords = [(float(label[i]) * w, float(label[i+1]) * h) for i in range(0, len(label), 2)]
    return abs_coords

def show_image_with_matplotlib(title, image):
    """ Matplotlib을 사용하여 이미지를 시각화하는 함수 """
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def extract_polygons_and_images(image, label_list):
    """ 폴리곤 내부의 이미지를 잘라내기 위한 마스크 생성 및 이미지 추출 """
    img_size = image.shape[:2]  # 이미지 크기 (height, width)
    
    cropped_images = []
    masks = []
    
    for label in label_list:
        label_split = label.split()
        coords = label_split[1:]  # 좌표 부분만 추출
        
        # 좌표를 절대 좌표로 변환
        obb_coords = convert_to_absolute(coords, img_size)
        polygon = np.array(obb_coords, dtype=np.int32)

        # 폴리곤에 맞는 마스크 생성
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [polygon], 255)
        cropped_img = cv2.bitwise_and(image, image, mask=mask)
        
        # 마스크와 잘라낸 이미지를 리스트에 저장
        masks.append(mask)
        cropped_images.append(cropped_img)
    
    return cropped_images, masks

def set_random_position(target_w, target_h, new_w, new_h):
    """ 타겟 이미지 내에서 무작위 위치를 설정하는 함수 """
    max_x_offset = max(0, target_w - new_w)
    max_y_offset = max(0, target_h - new_h)
    
    if max_x_offset > 0 and max_y_offset > 0:
        offset_x = random.randint(0, max_x_offset)
        offset_y = random.randint(0, max_y_offset)
    else:
        offset_x, offset_y = 0, 0  # 크기가 너무 큰 경우는 0, 0에 배치
    
    return offset_x, offset_y

def rotate_polygon(image, mask, angle):
    """ 폴리곤 마스크를 주어진 각도로 회전시키는 함수 """
    (h, w) = mask.shape[:2]
    center = (w // 2, h // 2)  # 중심 좌표
    M = cv2.getRotationMatrix2D(center, angle, 1.0)  # 회전 변환 행렬 생성
    
    # 이미지와 마스크 회전
    rotated_image = cv2.warpAffine(image, M, (w, h))
    rotated_mask = cv2.warpAffine(mask, M, (w, h), flags=cv2.INTER_NEAREST)  # 마스크는 이웃 보간법 사용
    
    return rotated_image, rotated_mask

def resize_and_paste_polygons(cropped_images, masks, target_img, scale=0.77/5, rotate_angle_range=(-180, 180)):
    """ 잘라낸 폴리곤 내부 이미지를 리사이즈하고 배경에 무작위로 배치 및 회전 적용 (랜덤 회전 각도) """
    # 최종 폴리곤 좌표를 저장할 리스트
    all_polygons_coords = []

    for cropped_img, mask in zip(cropped_images, masks):
        # 마스크에 맞는 영역을 잘라냄
        x, y, w, h = cv2.boundingRect(mask)
        cropped_polygon = cropped_img[y:y+h, x:x+w]
        mask_polygon = mask[y:y+h, x:x+w]

        # 이미지 리사이즈 (지정된 scale 적용)
        new_w = int(w * scale)
        new_h = int(h * scale)
        
        if new_w <= 1 or new_h <= 1:
            print("Image too small after resize, skipping.")
            continue
        
        # 폴리곤 이미지와 마스크 리사이즈
        resized_polygon = cv2.resize(cropped_polygon, (new_w, new_h), interpolation=cv2.INTER_AREA)
        resized_mask = cv2.resize(mask_polygon, (new_w, new_h), interpolation=cv2.INTER_AREA)
        
        # 랜덤 회전 각도 설정
        random_angle = random.randint(rotate_angle_range[0], rotate_angle_range[1])
        
        # 회전 각도 적용
        resized_polygon, resized_mask = rotate_polygon(resized_polygon, resized_mask, random_angle)

        # 위치 설정 (무작위 또는 다른 방식으로 설정 가능)
        offset_x, offset_y = set_random_position(target_img.shape[1], target_img.shape[0], new_w, new_h)

        # 최종 폴리곤 좌표를 계산 (4개 꼭짓점 좌표)
        polygon_coords = np.array([
            [offset_x, offset_y], 
            [offset_x + new_w, offset_y], 
            [offset_x + new_w, offset_y + new_h], 
            [offset_x, offset_y + new_h]
        ])
        all_polygons_coords.append(polygon_coords)  # 좌표 저장
        
        # 타겟 이미지에 폴리곤 이미지만 붙여넣기 (마스크로 영역 제한)
        region_of_interest = target_img[offset_y:offset_y+new_h, offset_x:offset_x+new_w]
        mask_inv = cv2.bitwise_not(resized_mask)
        
        target_bg = cv2.bitwise_and(region_of_interest, region_of_interest, mask=mask_inv)
        polygon_fg = cv2.bitwise_and(resized_polygon, resized_polygon, mask=resized_mask)
        
        combined = cv2.add(target_bg, polygon_fg)
        target_img[offset_y:offset_y+new_h, offset_x:offset_x+new_w] = combined
    
    return target_img, all_polygons_coords

# 예시 실행
target_img = np.ones((512, 512, 3), np.uint8) * 255  # 흰 배경
num_images = 5  # 사용할 이미지의 개수

def save_image_and_label(image, label_coords, image_num):
    """
    증강된 이미지를 저장하고, 라벨 파일도 함께 저장하는 함수
    :param image: (numpy array) 저장할 이미지
    :param label_coords: (리스트) 저장할 폴리곤 좌표 (각각의 좌표는 리스트)
    :param image_num: (int) 몇 번째 증강인지 나타내는 번호
    """
    image_save_path = f"./dataset3/images/augments{image_num}.png"
    label_save_path = f"./dataset3/labels/augments{image_num}.txt"

    # 이미지 저장
    cv2.imwrite(image_save_path, image)

    # 라벨 저장
    with open(label_save_path, 'w') as label_file:
        for polygon in label_coords:
            # 라벨 파일에 각 폴리곤의 좌표를 저장
            coords_str = ' '.join(map(str, polygon.flatten()))
            label_file.write(f"0 {coords_str}\n")  # 클래스 0으로 저장 (필요에 따라 변경 가능)

def augment_and_save_images(combined_dataset, target_dir_list ,num_samples, num_augments, target_size=(512, 512), scale=0.77/2.5, rotate_angle_range=(-180, 180)):
    """
    샘플링된 이미지에서 무작위 증강 데이터를 생성하고 저장하는 함수
    :param combined_dataset: (리스트) 이미지 경로와 라벨 파일 경로의 리스트
    :param num_samples: (정수) 샘플링할 이미지의 개수
    :param num_augments: (정수) 생성할 증강 이미지의 개수
    :param target_size: (튜플) 배경 이미지 크기 (default: 512x512)
    :param scale: (float) 리사이즈 스케일 (default: 0.77/2.5)
    :param rotate_angle_range: (튜플) 무작위 회전 각도 범위 (default: -180~180)
    """
    # 경로가 존재하지 않으면 생성
    os.makedirs("./dataset3/images/", exist_ok=True)
    os.makedirs("./dataset3/labels/", exist_ok=True)
    
    for augment_num in range(1, num_augments + 1):
        # 타겟 이미지로 사용할 이미지를 샘플링
        target_img_path = random.choice(target_dir_list)  # random.choice로 단일 경로 선택
        
        # 타겟 이미지를 불러오기
        target_img = cv2.imread(target_img_path)
        if target_img is None:
            print(f"Target image {target_img_path} not found.")
            continue
        
        # 타겟 이미지를 지정된 크기로 리사이즈 (필요한 경우)
        target_img = cv2.resize(target_img, target_size)

        all_polygons_coords = []  # 모든 폴리곤 좌표 저장 리스트
        
        # 데이터셋에서 무작위로 num_samples개의 이미지를 샘플링
        sampled_data = random.sample(combined_dataset, num_samples)
        
        for source_img_path, source_label_list in sampled_data:
            # 라벨 파일 읽기
            try:
                source_label_list = open(source_label_list).read().strip().split('\n')
            except FileNotFoundError:
                print(f"Label file {source_label_list} not found.")
                continue

            # 소스 이미지 불러오기
            source_img = cv2.imread(source_img_path)
            if source_img is None:
                print(f"Source image {source_img_path} not found.")
                continue

            # 폴리곤 내부의 이미지 및 마스크 추출
            cropped_images, masks = extract_polygons_and_images(source_img, source_label_list)

            # 리사이즈 후 배경에 삽입 및 랜덤 회전 적용
            target_img, polygon_coords = resize_and_paste_polygons(cropped_images, masks, target_img, scale=scale, rotate_angle_range=rotate_angle_range)
            all_polygons_coords.extend(polygon_coords)

        # 증강된 이미지와 라벨을 저장
        save_image_and_label(target_img, all_polygons_coords, augment_num)



num_samples = 2  # 샘플링할 이미지의 수
num_augments = 10  # 생성할 증강 데이터의 수
target_dir = target_dir
target_dir_list = [os.path.join(target_dir,file) for file in os.listdir(target_dir) if file.endswith('.png')]
# 증강 이미지 생성 및 저장
augment_and_save_images(combined_dataset, target_dir_list,num_samples, num_augments)

Label file sample/labels/train/OBJ00028_PS3_K3_NIA0078.txt not found.
Label file sample/labels/train/OBJ00028_PS3_K3_NIA0078.txt not found.
Label file sample/labels/train/OBJ00028_PS3_K3_NIA0078.txt not found.
Label file sample/labels/train/OBJ00028_PS3_K3_NIA0078.txt not found.


In [ ]:

def cluster_and_crop(image_path, k=3, size=1024, crop_num=43, threshold=0.7):
    # 이미지 로드 및 리사이즈
    img = Image.open(image_path)
    img = img.resize((size, size))
    
    # 이미지를 numpy 배열로 변환 (RGB 값 형태)
    img_np = np.array(img)
    
    # 이미지 데이터를 준비 (픽셀 수, 3) 형태로 변형
    img_data = img_np.reshape((-1, 3))
    img_data = np.float32(img_data)

    # K-Means 적용
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(img_data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # 클러스터링 결과를 이미지로 변환
    centers = np.uint8(centers)
    segmented_img = centers[labels.flatten()]
    segmented_img = segmented_img.reshape(img_np.shape)

    # 검은색에 가장 가까운 클러스터 찾기
    sea_cluster_index = np.argmin(np.linalg.norm(centers, axis=1))  # 가장 작은 RGB 값이 검은색
    sea_mask = (labels.reshape((img.shape[:2])) == sea_cluster_index).astype(np.uint8)
    
    return sea_mask

